# Imports and installations

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns #visualisation
from matplotlib import rcParams, figure
from scipy.stats import variation, mode, moment, linregress, norm
from scipy import stats
import matplotlib.patheffects as path_effects
import math
from pandas.plotting import parallel_coordinates
import matplotlib
import math

import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline 
sns.set(color_codes=True)

In [ ]:
!pip install bioinfokit
!pip install pingouin

# Utilities

In [ ]:
from scipy import stats

def spearmanr_ci(x,y,alpha=0.05):
    ''' calculate Spearman correlation along with the confidence interval using scipy and numpy
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Spearman's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''

    r, p = stats.spearmanr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return r, p, lo, hi

In [ ]:
MEASURES_MAP = {
    'FA': "fa_mean",
    'MD': "md_mean",
    'AD': "ad_mean",
    'RD': "rd_mean",
}

In [ ]:
out_path = Path(__file__).parent.parent.joinpath('results')

# Load data

In [ ]:
df = pd.read_excel("../data/subjects.xlsx")

## FA|MD|AD|RD ~ age 
No gender distinction

In [ ]:
from itertools import product

measures = ['FA', 'MD', 'AD', 'RD']
sub_cols = ['r', 'CI', 'P', 'P corr']
col_index = pd.MultiIndex.from_tuples(list(product(measures, sub_cols)))

columns = measures + ['Test']
structures = df['structure'].unique()

corr = pd.DataFrame(columns=col_index, index=structures)

In [ ]:
from scipy.stats import pearsonr, shapiro, spearmanr
from bioinfokit.analys import stat

alpha = 0.05

for structure in structures:
  print("Current structure: ", structure)
  for measure in measures:
    col_name = MEASURES_MAP[measure]

    structure_df = df[df['structure'] == structure]
    measure_avgs = structure_df[col_name]
    age = structure_df['age']

    r, p, lo, hi = spearmanr_ci(measure_avgs, age)
    ci = f'[{round(lo, 2)} {round(hi,2)}]'

    # add row to corr dataframe
    corr.loc[structure, (measure, slice(None))] = [round(r,2), ci, round(p,3), None]

In [ ]:
for measure in measures:
  # replace P values < 0.001 with <.001
  corr.loc[:, (measure, 'P')] = corr.loc[:, (measure, 'P')].apply(lambda v: '<.001' if v <= 0.001 else v)
  
  # remove leading 0 in P values
  corr.loc[:, (measure, 'P')] = corr.loc[:, (measure, 'P')].apply(lambda v: str(v).replace('0.', '.', 1))

In [ ]:
corr.to_excel(f'{out_path}/correlations_age_DTI.xlsx')

## Partial correlations

In [ ]:
# create base dataframe 
heading = ['r', '95% CI', 'p', 'p adj', 'test']
structures = df['structure'].unique()
measures = ['FA', 'MD', 'AD', 'RD']

pcor = pd.DataFrame(columns=heading, index=structures)

### FA ~ Age + AD

In [ ]:
from pingouin import partial_corr
from bioinfokit.analys import stat

alpha = 0.05
pcor_ad = pcor.copy()

for structure in structures:  
  structure_df = df[df['structure'] == structure]
  method = 'spearman'
  
  res = partial_corr(structure_df, 'age', 'fa_mean', 'ad_mean',
                              method=method)
  r = round(res['r'].values[0], 2)
  p = round(res['p-val'].values[0], 3)
  ci = res['CI95%'].values[0]

  # add row to corr dataframe
  pcor_ad.loc[structure, :] = np.array([r, ci, p, None, method], dtype='object')

In [ ]:
from statsmodels.stats.multitest import multipletests

pvals = pcor_ad['p']
reject_list, corrected_p_vals = multipletests(pvals, method='fdr_bh', alpha=0.05)[:2]

# format raw P values
p_form_raw = ['<.001' if p < 0.001 else str(round(p, 3)) for p in pvals]
p_form_raw = list(map(lambda p: p.replace('0.', '.'), p_form_raw))       

pcor_ad['p'] = p_form_raw

In [ ]:
pcor_ad.to_excel(f'{out_path}/partial_correlations_FA_age_AD.xlsx')

### FA ~ Age + RD

In [ ]:
from pingouin import partial_corr
from bioinfokit.analys import stat

alpha = 0.05
pcor_rd = pcor.copy()

for structure in structures:  
  structure_df = df[df['structure'] == structure]
  method = 'spearman'
  
  res = partial_corr(structure_df, 'age', 'fa_mean', 'rd_mean',
                              method=method)
  r = round(res['r'].values[0], 2)
  p = round(res['p-val'].values[0], 3)
  ci = res['CI95%'].values[0]

  # add row to corr dataframe
  pcor_rd.loc[structure, :] = np.array([r, ci, p, None, method], dtype='object')

In [ ]:
from statsmodels.stats.multitest import multipletests

pvals = pcor_rd['p']
reject_list, corrected_p_vals = multipletests(pvals, method='fdr_bh', alpha=0.05)[:2]

# format raw P values
p_form_raw = ['<.001' if p < 0.001 else str(round(p, 3)) for p in pvals]
p_form_raw = list(map(lambda p: p.replace('0.', '.'), p_form_raw))       

pcor_rd['p'] = p_form_raw

In [ ]:
pcor_rd.to_excel(f'{out_path}/partial_correlations_FA_age_RD.xlsx')

### FA ~ Age + MD

In [ ]:
from pingouin import partial_corr
from bioinfokit.analys import stat

alpha = 0.05
pcor_md = pcor.copy()

for structure in structures:  
  structure_df = df[df['structure'] == structure]
  method = 'spearman'
  
  res = partial_corr(structure_df, 'age', 'fa_mean', 'md_mean',
                              method=method)
  r = round(res['r'].values[0], 2)
  p = round(res['p-val'].values[0], 3)
  ci = res['CI95%'].values[0]

  # add row to corr dataframe
  pcor_md.loc[structure, :] = np.array([r, ci, p, None, method], dtype='object')

In [ ]:
from statsmodels.stats.multitest import multipletests

pvals = pcor_md['p']
reject_list, corrected_p_vals = multipletests(pvals, method='fdr_bh', alpha=0.05)[:2]

# format raw P values
p_form_raw = ['<.001' if p < 0.001 else str(round(p, 3)) for p in pvals]
p_form_raw = list(map(lambda p: p.replace('0.', '.'), p_form_raw))       

pcor_md['p'] = p_form_raw

In [ ]:
pcor_md.to_excel(f'{out_path}/partial_correlations_FA_age_MD.xlsx')